# VR ASSIGNMENT 2 - Q2b - IMT2019092

In this question I have implemented Bike vs Horses classification using the dataset provided. I have implemented it using the Bag of visual words approach. I have extracted image descriptors using SIFT. Then used K-means to form clusters and extracted bag of visual words. Finally applied Tfidf transformer and tried to train using Logistic regression, SVM and KNN models.


In [73]:
import cv2
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.model_selection import train_test_split
import pickle

In [74]:
dir_path = '../input/bikesandhorses/data/'
categories = os.listdir(dir_path)
categories

In [75]:
#Reading Images
label = -1
images = []
labels = []
for category in categories:
    label+=1
    path = dir_path +'/'+category
    for img in os.listdir(path):
        images.append(cv2.imread(path + "/" + img))
        labels.append(label)

In [76]:
#Train Test Split
train_imgs, test_imgs, trainLabels, testLabels = train_test_split(images,labels,train_size=0.8, random_state=33, stratify=labels)

In [77]:
#Extracting Descriptors using SIFT
all_descriptors = np.asarray([])
extractor = cv2.SIFT_create()
for img in train_imgs:
    kps, descriptors = extractor.detectAndCompute(img, None)
    if type(descriptors)==np.ndarray:
        if all_descriptors.shape[0]==0:
            all_descriptors = descriptors
        else:
            all_descriptors = np.concatenate((all_descriptors, descriptors), axis=0)

In [78]:
#Running KMeans with 200 clusters
kmeans = KMeans(n_clusters = 200)
kmeans.fit(all_descriptors)

In [79]:
#Function to get bag of visual words
def get_bovw(imgs):
    bovw = []
    extractor = cv2.SIFT_create()
    for img in imgs:
        bokps = [0]*500
        kps, descriptors = extractor.detectAndCompute(img, None)
        if type(descriptors) == np.ndarray:
            pred = kmeans.predict(descriptors)
            for e in pred:
                bokps[e] = bokps[e] + 1
        bovw.append(bokps)

    features = np.asarray(bovw)
    return features

In [80]:
pickle.dump(kmeans, open('kmfile.sav', 'wb'))
#vocab = pickle.load(open('kmfile.sav', 'rb'))

In [81]:
#Applying tfidf transformation
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(sublinear_tf = True)
train_features = get_bovw(train_imgs)
tfidf_features = tfidf_transformer.fit_transform(train_features)
test_features = get_bovw(test_imgs)
tfidf_test_features = tfidf_transformer.transform(test_features)

## Without TFIDF

### SVC

In [82]:
model = svm.SVC(C=1,kernel='linear')
model.fit(train_features, trainLabels)

In [83]:
accuracy = model.score(test_features, testLabels)
print(accuracy)

In [84]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
plot_confusion_matrix(model, test_features, testLabels)
plt.show()

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=7)
model.fit(train_features, trainLabels)

In [86]:
accuracy = model.score(test_features, testLabels)
print(accuracy)

In [87]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
plot_confusion_matrix(model, test_features, testLabels)
plt.show()

### Logistic Regression

In [88]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(train_features, trainLabels)

In [89]:
accuracy = model.score(test_features, testLabels)
print(accuracy)

In [90]:
plot_confusion_matrix(model, test_features, testLabels)
plt.show()

In [ ]:
### Testing with random images

In [91]:
test_path = '../input/bikesandhorses-test/test'
test_images = []
labels = [0,0,0,1,1,1]
for img in os.listdir(test_path):
    test_images.append(cv2.imread(test_path + "/" + img))
plt.imshow(cv2.cvtColor(test_images[-1], cv2.COLOR_RGB2BGR))

In [92]:
test_labels = get_bovw(test_images)
print(model.predict(test_labels))

## With TFIDF

### SVC

In [93]:
model = svm.SVC(C=1,kernel='linear')
model.fit(tfidf_features, trainLabels)

In [94]:
accuracy = model.score(tfidf_test_features, testLabels)
print(accuracy)

In [95]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
plot_confusion_matrix(model, tfidf_test_features, testLabels)
plt.show()

### KNN

In [96]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=9)
model.fit(tfidf_features, trainLabels)

In [97]:
accuracy = model.score(tfidf_test_features, testLabels)
print(accuracy)

In [98]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
plot_confusion_matrix(model, tfidf_test_features, testLabels)
plt.show()

### Logistic Regression

In [99]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(tfidf_features, trainLabels)

In [100]:
accuracy = model.score(tfidf_test_features, testLabels)
print(accuracy)

In [101]:
plot_confusion_matrix(model, tfidf_test_features, testLabels)
plt.show()

### Testing with random images

In [104]:
model = svm.SVC(C=1,kernel='linear')
model.fit(tfidf_features, trainLabels)
test_path = '../input/bikesandhorses-test/test'
test_images = []
labels = [0,0,0,1,1,1]
for img in os.listdir(test_path):
    test_images.append(cv2.imread(test_path + "/" + img))
plt.imshow(cv2.cvtColor(test_images[-1], cv2.COLOR_RGB2BGR))

In [105]:
test_labels = get_bovw(test_images)
tfidf_test_labels = tfidf_transformer.transform(test_labels)
print(model.predict(tfidf_test_labels))